In [58]:
from datetime import datetime, timedelta
from pathlib import Path
import os
import sys
import numpy as np
import pandas as pd
import re
import AMBRA_Backups
from bs4 import BeautifulSoup
import AMBRA_Utils
from redcap_funcs import comp_schema_cap_db, get_project_schema

db_name = 'TESTED'
db = AMBRA_Backups.database.Database(db_name)
project_name = 'TESTED Data Collection'
project = AMBRA_Backups.redcap_funcs.get_redcap_project(project_name) 

def q(query, record=None):
    if any(substring in query for substring in ['UPDATE', 'INSERT', 'DELETE']):
        return db.run_insert_query(query, record)
    elif 'SELECT' in query:
        return pd.DataFrame(db.run_select_query(query, record, column_names=True))
    else:
        print('You dont have UPDATE, INSERT, DELETE or SELECT in your query my guy')

def display_md(df):
    return print(df.to_markdown())

In [45]:
crf_name = 'mr'

schema_questions = pd.DataFrame(db.run_select_query("""SELECT question_text, redcap_variable FROM CRF_Schema_RedCap
                                WHERE crf_name = %s AND question_text IS NOT NULL""", [crf_name], column_names=True))
schema_questions['variable-value'] = schema_questions['redcap_variable']+schema_questions['question_text']

api_questions = pd.DataFrame(project.export_metadata())
api_questions = api_questions[api_questions['form_name'] == crf_name]
field_names = pd.DataFrame(project.export_field_names())
field_names.rename(columns={'original_field_name': 'field_name'}, inplace=True)
# api_questions = pd.merge(field_names, api_questions, on='field_name')
api_questions = pd.merge(api_questions, field_names, on='field_name', how='left')
api_questions

,field_name,form_name,section_header,field_type,field_label,select_choices_or_calculations,field_note,text_validation_type_or_show_slider_number,text_validation_min,text_validation_max,identifier,branching_logic,required_field,custom_alignment,question_number,matrix_group_name,matrix_ranking,field_annotation,choice_value,export_field_name
0,q2001,mr,,radio,Time point,"1, Baseline SOC | 2, Follow-Up | 3, Unscheduled",,,,,,,,RH,,,,"@DEFAULT=""1""",,q2001
1,q2002,mr,,checkbox,Scan Type,"1, MRI | 2, MRA | 3, MRP | 4, Others",,,,,,,,RH,,,,,1,q2002___1
2,q2002,mr,,checkbox,Scan Type,"1, MRI | 2, MRA | 3, MRP | 4, Others",,,,,,,,RH,,,,,2,q2002___2
3,q2002,mr,,checkbox,Scan Type,"1, MRI | 2, MRA | 3, MRP | 4, Others",,,,,,,,RH,,,,,3,q2002___3
4,q2002,mr,,checkbox,Scan Type,"1, MRI | 2, MRA | 3, MRP | 4, Others",,,,,,,,RH,,,,,4,q2002___4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,q2129,mr,,text,Signature of Central Reader,,,,,,,,,RH,,,,,,q2129
367,q2131,mr,,notes,Reviewer's general comments,,,,,,,,,RH,,,,,,q2131
368,mr_status,mr,"<div class=""rich-text-field-label""><p><strong>...",radio,CRF status,"1, Unread | 2, Query generated | 3, Read - Unv...",,,,,,,,,,,,@IF([user-role-label] = 'Admin' or user-role-l...,,mr_status
369,mri_comments,mr,,notes,CRP comments,,,,,,,,,,,,,@IF([user-role-label] = 'Admin' or user-role-l...,,mri_comments


In [63]:

patient_name = '11003'
patient_id = '6037'
instance = 1
verified = ''

form_df = AMBRA_Backups.redcap_funcs.get_form_df(project, patient_name, crf_name, instance)
# if form_df.empty: # if empty, means there is no live data for this patient and a deleted log should appear later
#     continue 
# crf_id = db.run_insert_query(f"""INSERT INTO CRF_RedCap (id_patient, crf_name, instance, verified, deleted) VALUES 
#                                 (%s, %s, {'NULL' if instance is None else instance}, %s, %s)""", [patient_id, crf_name, verified, deleted])

irr_cols = 3 if form_df.columns[1] == 'redcap_repeat_instrument' else 1 # number of irrelevant fields ie. record_id, redcap_repeat_instrument, redcap_repeat_instance
form_df = form_df[form_df.columns[irr_cols:]]
form_df = form_df.melt(var_name='redcap_variable')
form_df.loc[form_df['redcap_variable'].str.contains('___'), 'redcap_variable'] = form_df['redcap_variable']+')'
form_df.loc[form_df['redcap_variable'].str.contains('___'), 'redcap_variable'] = form_df['redcap_variable'].str.replace('___', '(')

def under_to_neg(row):
    redcap_variable = row['redcap_variable']
    if '(' in redcap_variable and ')' in redcap_variable:
        option = redcap_variable.split('(')[1].split(')')[0]
        if option 
            
            

form_df[form_df['redcap_variable'].str.contains('2018')]

,redcap_variable,value
66,q2018a,1
117,q2018(0),0
118,q2018(1),0
119,q2018(2),0
120,q2018(3),0
121,q2018(_10),1


In [46]:
# 1. record initialization
patient_name = '11003'
crf_name = 'mr'
form_df = pd.DataFrame(project.export_records(records=[patient_name], forms=[crf_name]))
form_df

,record_id,redcap_repeat_instrument,redcap_repeat_instance,q2001,q2002___1,q2002___2,q2002___3,q2002___4,q2003___1,q2003___2,...,q2119,q2121,q2122,q2124,q2129,q2131,mr_status,mri_comments,mr_init,mr_complete
0,11003,,,3,1,0,0,0,1,1,...,,,,,MGS,Only DWI sequence\r\n,4,"""Prior infarct seen on image?"" unanswered; JJJ...",JJJ,2


In [59]:
# 2. insert from logs


s = AMBRA_Backups.redcap_funcs.grab_logs(db, project, only_record_logs=False, start_date=datetime(2000, 1, 1))
s = pd.DataFrame(s)
sub = s[s['record'] == '11003']
display_md(sub)

|      | timestamp        | username   | action              | details                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |   record |
|-----:|:-----------------|:-----------|:--------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------

In [44]:
comp_schema_cap_db(db_name, project_name)

/Users/naderdl/Documents/AMBRA_Backups/AMBRA_Backups/redcap_funcs.py:283: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(row['field_label'], 'html.parser')
/Users/naderdl/Documents/AMBRA_Backups/AMBRA_Backups/redcap_funcs.py:283: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(row['field_label'], 'html.parser')
/Users/naderdl/Documents/AMBRA_Backups/AMBRA_Backups/redcap_funcs.py:283: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(row['field_label'], 'html.parser')



-------------------mr-------------------

The following CRF_Data_RedCap.redcap_variable's are not in CRF_schema_RedCap.redcap_variable's:
['q2018(-10)', 'q2029_mr(-10)', 'q1036_mr(3)']


The following api-metadata question_text's are not in CRF_Schema_RedCap.question_text's:
{redcap_variable : question_text}

{'q2018(-10)': 'White matter hyperintensity (WMH) Periventricular?', 'q2029_mr(-10)': 'White matter hyperintensity (WMH) deep white matter (DWM)?'}




Exception: Please handle the above discrepancies